In [ ]:
import re
import numpy as np
import pandas as pd

from pathlib import Path



In [ ]:
data_path = Path("../data")

# Note the Capital Letters and correct filenames
TRAIN_PATH = data_path / "Train.csv"
TEST_PATH  = data_path / "Test.csv"
SUB_PATH   = data_path / "SampleSubmission.csv"



True
True


In [8]:
print(data_path.exists())
print(TRAIN_PATH.exists())

True
True


In [9]:
train = pd.read_csv(TRAIN_PATH)
print("train shape:", train.shape)




train shape: (29132, 29)


In [10]:
test  = pd.read_csv(TEST_PATH)
print("test  shape:", test.shape)


test  shape: (10000, 29)


In [11]:
sub   = pd.read_csv(SUB_PATH)
print("sub   shape:", sub.shape)

sub   shape: (210000, 2)


In [12]:

# verifieons qu'on a les memes colonnes dans train et test
assert list(train.columns) == list(test.columns), "Train/Test: colonnes différentes (problème de schéma)."

# verifieons qu'on a pas de valeurs manquantes
assert train["ID"].isna().sum() == 0
assert test["ID"].isna().sum() == 0



In [13]:
train.head(3)

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
PROFILE_COLS = [
    "ID", "join_date", "sex", "marital_status", "birth_year",
    "branch_code", "occupation_code", "occupation_category_code"
]

PRODUCT_COLS = [c for c in train.columns if c not in PROFILE_COLS]

print("Nb colonnes profil :", len(PROFILE_COLS))
print("Nb colonnes produits:", len(PRODUCT_COLS))
print("Produits:", PRODUCT_COLS)




Nb colonnes profil : 8
Nb colonnes produits: 21
Produits: ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']


Ici, la particularité est que la cible n’est pas donnée directement : dans le fichier Test,un produit que le client possède réellement  a été masqué en remplaçant un 1 par 0 dans les 21 produits.
Le but du modèle est donc de retrouver quel produit a été caché.
Le “masque”, c’est juste cette opération : prendre un produit détenu (1) et le mettre à 0 pour simuler un “produit manquant”.
On doit le faire aussi sur Train (pour créer un apprentissage supervisé), sinon le modèle apprend sur des paniers “complets” et il sera en décalage avec Test où un produit est manquant.

In [15]:
train_basket = train[PRODUCT_COLS].sum(axis=1)
test_basket  = test[PRODUCT_COLS].sum(axis=1)

train_basket.name = "basket_size_train"
test_basket.name  = "basket_size_test"


In [18]:
print("Basket size (train) - describe:\n", train_basket.describe())
print("\nBasket size (test)  - describe:\n", test_basket.describe())

mean_gap = round(float(train_basket.mean() - test_basket.mean()),2)
print("\nMean gap (train - test):", mean_gap)


Basket size (train) - describe:
 count    29132.000000
mean         2.277667
std          0.602677
min          2.000000
25%          2.000000
50%          2.000000
75%          2.000000
max         14.000000
Name: basket_size_train, dtype: float64

Basket size (test)  - describe:
 count    10000.00000
mean         1.28530
std          0.59088
min          1.00000
25%          1.00000
50%          1.00000
75%          1.00000
max          7.00000
Name: basket_size_test, dtype: float64

Mean gap (train - test): 0.99


Le résultat Mean gap (train - test): 0.99 signifie qu’en moyenne, les clients du Test ont ~1 produit “1” de moins que ceux du Train. C’est exactement le pattern attendu car un produit  a été masqué (remplacer un 1 par 0) dans Test.

In [19]:
print("\nMin train basket:", int(train_basket.min()), " / Min test basket:", int(test_basket.min()))
print("Nb train basket_size==0:", int((train_basket == 0).sum()))
print("Nb test  basket_size==0:", int((test_basket  == 0).sum()))

assert (train_basket >= 1).all(), "Train: clients avec 0 produit → surprenant (à investiguer)."
assert (test_basket  >= 1).all(), "Test: clients avec 0 produit → incompatible avec masking '1 produit'."



Min train basket: 2  / Min test basket: 1
Nb train basket_size==0: 0
Nb test  basket_size==0: 0


In [20]:
print("Répartition basket_size (train) - top 15:")
print(train_basket.value_counts().sort_index().head(15))



Répartition basket_size (train) - top 15:
basket_size_train
2     22732
3      5169
4       916
5       215
6        71
7        22
8         5
9         1
14        1
Name: count, dtype: int64


In [21]:
print("\nRépartition basket_size (test)  - top 15:")
print(test_basket.value_counts().sort_index().head(15))




Répartition basket_size (test)  - top 15:
basket_size_test
1    7685
2    1921
3     287
4      76
5      26
6       4
7       1
Name: count, dtype: int64


In [22]:
print("\nNb lignes train avec basket_size == 1:", int((train_basket == 1).sum()))
print("Nb lignes test  avec basket_size == 1:", int((test_basket  == 1).sum()))



Nb lignes train avec basket_size == 1: 0
Nb lignes test  avec basket_size == 1: 7685


In [23]:
# La colonne est souvent nommée exactement comme ça
assert "ID X PCODE" in sub.columns, "SampleSubmission: colonne 'ID X PCODE' manquante."
assert "Label" in sub.columns, "SampleSubmission: colonne 'Label' manquante."




In [24]:
tmp = sub["ID X PCODE"].astype(str).str.split(" X ", expand=True)
tmp.columns = ["ID", "PCODE"]

print("Nb IDs dans submission:", tmp["ID"].nunique())
print("Nb produits distincts:", tmp["PCODE"].nunique())
print("Répartition nb lignes par ID (doit être constant):")
print(tmp.groupby("ID").size().value_counts().head())


Nb IDs dans submission: 10000
Nb produits distincts: 21
Répartition nb lignes par ID (doit être constant):
21    10000
Name: count, dtype: int64


In [25]:

# Assertions structurelles
assert tmp["ID"].nunique() == test["ID"].nunique(), "Submission: nb IDs != nb IDs test."
assert set(tmp["PCODE"].unique()) == set(PRODUCT_COLS), "Submission: produits != colonnes produits du dataset."
assert (tmp.groupby("ID").size() == len(PRODUCT_COLS)).all(), "Submission: chaque ID doit avoir exactement 21 lignes."

In [27]:
def extract_year_from_join_date(s: str) -> float:
    """
    Extraction  de l'année (on ne fait PAS confiance au jour/mois car format ambigu).
    Retourne np.nan si pas d'année.
    """
    if pd.isna(s):
        return np.nan
    m = re.search(r"(\d{4})", str(s))
    return float(m.group(1)) if m else np.nan


def assert_products_binary(df: pd.DataFrame, product_cols: list[str], df_name: str) -> None:
    bad = {}
    for c in product_cols:
        vals = set(pd.unique(df[c].dropna()))
        if not vals.issubset({0, 1}):
            bad[c] = sorted(list(vals))[:10]
    assert len(bad) == 0, f"{df_name}: colonnes produits non binaires détectées: {bad}"


def unknown_categories_report(train_df: pd.DataFrame, test_df: pd.DataFrame, cat_cols: list[str]) -> pd.DataFrame:
    rows = []
    for col in cat_cols:
        tr = set(train_df[col].dropna().astype(str).unique())
        te = set(test_df[col].dropna().astype(str).unique())
        unseen = sorted(list(te - tr))
        rows.append({
            "col": col,
            "train_unique": len(tr),
            "test_unique": len(te),
            "unseen_in_train_count": len(unseen),
            "unseen_examples": unseen[:10]
        })
    return pd.DataFrame(rows).sort_values("unseen_in_train_count", ascending=False)


def age_at_join_report(df: pd.DataFrame, df_name: str) -> pd.Series:
    join_year = df["join_date"].apply(extract_year_from_join_date)
    birth_year = pd.to_numeric(df["birth_year"], errors="coerce")
    age = join_year - birth_year
    age = age.dropna()
    print(f"{df_name}: join_year NaN:", int(join_year.isna().sum()))
    return age


In [38]:


def join_date_audit(df: pd.DataFrame, name: str) -> pd.DataFrame:
    s = df["join_date"]
    join_year = s.apply(extract_year_from_join_date)

    # Parsing "naïf" des 2 premiers champs pour mesurer l'ambiguïté
    tmp = s.dropna().astype(str).str.split("/", expand=True)
    tmp.columns = ["part1", "part2", "part3"]
    tmp["part1"] = pd.to_numeric(tmp["part1"], errors="coerce")
    tmp["part2"] = pd.to_numeric(tmp["part2"], errors="coerce")
    tmp["year"]  = pd.to_numeric(tmp["part3"], errors="coerce")

    ambiguous = ((tmp["part1"] <= 12) & (tmp["part2"] <= 12)).mean()  # jour/mois indiscernables
    day_gt12  = (tmp["part1"] > 12).mean()  # indique souvent "jour d'abord"
    month_gt12 = (tmp["part2"] > 12).mean() # indiquerait "mois d'abord" (rare ici)

    report = pd.DataFrame([{
        "dataset": name,
        "join_date_missing": int(s.isna().sum()),
        "join_year_missing": int(join_year.isna().sum()),
        "join_year_min": float(np.nanmin(join_year.values)),
        "join_year_max": float(np.nanmax(join_year.values)),
        "ambiguous_day_month_share": float(ambiguous),
        "part1_gt12_share": float(day_gt12),
        "part2_gt12_share": float(month_gt12),
    }])
    return report, join_year

train_join_report, train_join_year = join_date_audit(train, "train")
test_join_report,  test_join_year  = join_date_audit(test,  "test")

pd.concat([train_join_report, test_join_report], ignore_index=True)


,dataset,join_date_missing,join_year_missing,join_year_min,join_year_max,ambiguous_day_month_share,part1_gt12_share,part2_gt12_share
0,train,2,2,2010.0,2020.0,0.999863,0.000137,0.0
1,test,1,1,2010.0,2020.0,0.999900,0.000100,0.0


In [33]:
# 1) Missing values
missing_train = train.isna().mean().sort_values(ascending=False)
missing_test  = test.isna().mean().sort_values(ascending=False)

print("Top missing (train):\n", missing_train.head(10))
print("\nTop missing (test):\n", missing_test.head(10))



Top missing (train):
 join_date                   0.000069
ID                          0.000000
sex                         0.000000
marital_status              0.000000
birth_year                  0.000000
branch_code                 0.000000
occupation_code             0.000000
occupation_category_code    0.000000
P5DA                        0.000000
RIBP                        0.000000
dtype: float64

Top missing (test):
 join_date                   0.0001
ID                          0.0000
sex                         0.0000
marital_status              0.0000
birth_year                  0.0000
branch_code                 0.0000
occupation_code             0.0000
occupation_category_code    0.0000
P5DA                        0.0000
RIBP                        0.0000
dtype: float64


In [29]:

# 2) Produits binaires (assertions)
assert_products_binary(train, PRODUCT_COLS, "train")
assert_products_binary(test,  PRODUCT_COLS, "test")



In [30]:
# 3) join_date -> join_year (robuste) + âge à l'entrée
age_train = age_at_join_report(train, "train")
age_test  = age_at_join_report(test,  "test")

print("\nAge at join (train):\n", age_train.describe())
print("\nAge at join (test):\n",  age_test.describe())



train: join_year NaN: 2
test: join_year NaN: 1

Age at join (train):
 count    29130.000000
mean        38.201716
std          9.251233
min          8.000000
25%         31.000000
50%         38.000000
75%         45.000000
max         80.000000
dtype: float64

Age at join (test):
 count    9999.000000
mean       38.028303
std         9.202317
min        17.000000
25%        31.000000
50%        37.000000
75%        45.000000
max        87.000000
dtype: float64


In [31]:
# Compteurs d'anomalies (tu ajusteras les seuils selon la réalité métier)
print("\nAnomalies âge (train):")
print("  age < 16:", int((age_train < 16).sum()))
print("  age < 18:", int((age_train < 18).sum()))
print("  age > 90:", int((age_train > 90).sum()))

print("\nAnomalies âge (test):")
print("  age < 16:", int((age_test < 16).sum()))
print("  age < 18:", int((age_test < 18).sum()))
print("  age > 90:", int((age_test > 90).sum()))




Anomalies âge (train):
  age < 16: 2
  age < 18: 7
  age > 90: 0

Anomalies âge (test):
  age < 16: 0
  age < 18: 3
  age > 90: 0


In [32]:
# 4) Catégories inconnues en test (problème prod classique)
CAT_COLS = ["sex", "marital_status", "branch_code", "occupation_code", "occupation_category_code"]
unk_df = unknown_categories_report(train, test, CAT_COLS)
unk_df


,col,train_unique,test_unique,unseen_in_train_count,unseen_examples
3,occupation_code,233,187,9,"[0FOI, 0ZND, 8CHJ, 93OJ, 9F96, BIA0, E2MJ, HSI..."
1,marital_status,8,8,1,[F]
0,sex,2,2,0,[]
2,branch_code,15,15,0,[]
4,occupation_category_code,6,6,0,[]


In [34]:
# 2) Outliers
outliers_train = train.isin([np.inf, -np.inf]).sum().sort_values(ascending=False)
outliers_test  = test.isin([np.inf, -np.inf]).sum().sort_values(ascending=False)

print("Top outliers (train):\n", outliers_train.head(10))
print("\nTop outliers (test):\n", outliers_test.head(10))


    

Top outliers (train):
 ID                          0
join_date                   0
sex                         0
marital_status              0
birth_year                  0
branch_code                 0
occupation_code             0
occupation_category_code    0
P5DA                        0
RIBP                        0
dtype: int64

Top outliers (test):
 ID                          0
join_date                   0
sex                         0
marital_status              0
birth_year                  0
branch_code                 0
occupation_code             0
occupation_category_code    0
P5DA                        0
RIBP                        0
dtype: int64


In [35]:
# 3) Duplicates
duplicates_train = train.duplicated().sum()
duplicates_test  = test.duplicated().sum()

print("Duplicates (train):", duplicates_train)
print("Duplicates (test):", duplicates_test)


Duplicates (train): 0
Duplicates (test): 0


In [36]:

# 4) Data types
data_types_train = train.dtypes.value_counts()
data_types_test  = test.dtypes.value_counts()

print("Data types (train):\n", data_types_train)
print("\nData types (test):\n", data_types_test)


Data types (train):
 int64     22
object     7
Name: count, dtype: int64

Data types (test):
 int64     22
object     7
Name: count, dtype: int64


In [39]:
def unseen_categories_report(train_df: pd.DataFrame, test_df: pd.DataFrame, cat_cols: list[str]) -> pd.DataFrame:
    rows = []
    for col in cat_cols:
        tr = set(train_df[col].dropna().astype(str).unique())
        te = set(test_df[col].dropna().astype(str).unique())
        unseen = sorted(list(te - tr))
        rows.append({
            "col": col,
            "train_unique": len(tr),
            "test_unique": len(te),
            "unseen_in_train_count": len(unseen),
            "unseen_examples": unseen[:10]
        })
    return pd.DataFrame(rows).sort_values("unseen_in_train_count", ascending=False)

unseen_df = unseen_categories_report(train, test, CAT_COLS)
unseen_df


,col,train_unique,test_unique,unseen_in_train_count,unseen_examples
3,occupation_code,233,187,9,"[0FOI, 0ZND, 8CHJ, 93OJ, 9F96, BIA0, E2MJ, HSI..."
1,marital_status,8,8,1,[F]
0,sex,2,2,0,[]
2,branch_code,15,15,0,[]
4,occupation_category_code,6,6,0,[]


In [40]:
def age_audit(df: pd.DataFrame, join_year: pd.Series, name: str) -> pd.DataFrame:
    birth_year = pd.to_numeric(df["birth_year"], errors="coerce")
    age = join_year - birth_year

    report = pd.DataFrame([{
        "dataset": name,
        "age_missing": int(age.isna().sum()),
        "age_min": float(np.nanmin(age.values)),
        "age_p1": float(np.nanpercentile(age.values, 1)),
        "age_p50": float(np.nanpercentile(age.values, 50)),
        "age_p99": float(np.nanpercentile(age.values, 99)),
        "age_max": float(np.nanmax(age.values)),
        "age_lt_18": int((age < 18).sum()),
        "age_gt_90": int((age > 90).sum()),
    }])
    return report, age

train_age_report, train_age = age_audit(train, train_join_year, "train")
test_age_report,  test_age  = age_audit(test,  test_join_year,  "test")

pd.concat([train_age_report, test_age_report], ignore_index=True)


,dataset,age_missing,age_min,age_p1,age_p50,age_p99,age_max,age_lt_18,age_gt_90
0,train,2,8.0,21.0,38.0,59.0,80.0,7,0
1,test,1,17.0,21.0,37.0,59.0,87.0,3,0


In [41]:
def assert_products_binary(df: pd.DataFrame, product_cols: list[str], name: str) -> None:
    bad = {}
    for c in product_cols:
        vals = set(pd.unique(df[c].dropna()))
        if not vals.issubset({0, 1}):
            bad[c] = sorted(list(vals))[:10]
    assert len(bad) == 0, f"{name}: colonnes produits non binaires détectées: {bad}"

assert_products_binary(train, PRODUCT_COLS, "train")
assert_products_binary(test,  PRODUCT_COLS, "test")

print("OK: tous les produits sont strictement binaires (0/1).")


OK: tous les produits sont strictement binaires (0/1).


In [46]:
def clean_join_year(df: pd.DataFrame, join_year_fill: float) -> pd.DataFrame:
    out = df.copy()
    out["join_year"] = out["join_date"].apply(extract_year_from_join_date)
    out["join_year_missing"] = out["join_year"].isna().astype(int)
    out["join_year"] = out["join_year"].fillna(join_year_fill)
    return out

def clean_age_at_join(df: pd.DataFrame, min_age: int = 18, max_age: int = 90) -> pd.DataFrame:
    out = df.copy()
    birth_year = pd.to_numeric(out["birth_year"], errors="coerce")
    age = out["join_year"] - birth_year

    out["age_at_join_raw"] = age
    out["age_missing"] = age.isna().astype(int)

    # clip + flag
    age_clipped = age.clip(lower=min_age, upper=max_age)
    out["age_was_clipped"] = ((age_clipped != age) & (~age.isna())).astype(int)
    out["age_at_join"] = age_clipped

    return out

def clean_categoricals_with_unknown(train_df: pd.DataFrame, df: pd.DataFrame, cat_cols: list[str]) -> pd.DataFrame:
    out = df.copy()
    for col in cat_cols:
        out[col] = out[col].astype(str)
        out[col] = out[col].replace({"nan": "UNKNOWN"})
        known = set(train_df[col].dropna().astype(str).unique())
        out[col] = out[col].where(out[col].isin(known), "UNKNOWN")
    return out


In [48]:
# join_year_fill: médiane train (robuste)
join_year_fill = float(pd.Series(train["join_date"].apply(extract_year_from_join_date)).dropna().median())

train_clean = train.copy()
test_clean  = test.copy()

train_clean = clean_join_year(train_clean, join_year_fill)
test_clean  = clean_join_year(test_clean,  join_year_fill)

train_clean = clean_age_at_join(train_clean, min_age=18, max_age=90)
test_clean  = clean_age_at_join(test_clean,  min_age=18, max_age=90)

train_clean = clean_categoricals_with_unknown(train_clean, train_clean, CAT_COLS)
test_clean  = clean_categoricals_with_unknown(train_clean, test_clean,  CAT_COLS)

# Mini bilan
def summarize_cleaning(df: pd.DataFrame, name: str) -> pd.DataFrame:
    return pd.DataFrame([{
        "dataset": name,
        "join_year_missing": int(df["join_year_missing"].sum()),
        "age_missing": int(df["age_missing"].sum()),
        "age_was_clipped": int(df["age_was_clipped"].sum()),
        "unknown_marital_status": int((df["marital_status"] == "UNKNOWN").sum()),
        "unknown_occupation_code": int((df["occupation_code"] == "UNKNOWN").sum()),
    }])

pd.concat([summarize_cleaning(train_clean, "train_clean"),
           summarize_cleaning(test_clean,  "test_clean")], ignore_index=True)


,dataset,join_year_missing,age_missing,age_was_clipped,unknown_marital_status,unknown_occupation_code
0,train_clean,2,0,7,0,0
1,test_clean,1,0,3,1,10
